In [ ]:
### SET DRIVE PATH ###
drive = "/SETDRIVEPATH/data_lsd_trip/"

In [ ]:
## the actual algorithms are in tcspc_functions.py
%run tcspc_functions.py

initial_guess_coeffs = np.array([42.4, # musp500
                                 0.62, # fray 
                                 1.0], # bmie
                                dtype=float)
_nm = np.arange(600,841,20)
_nm_ext = np.arange(450,1000,10)

water_ref1 = pd.read_csv(
    drive + "/data_calibration/literature/segelstein81.csv", 
    header=3, sep="\t")

irf_rho = 47e-3 # meters
medium_rho = 20e-3 # meters
sig_start = 0.5
measurement_duration = 25e-9 # sec

In [ ]:
## Processing
## 1st batch
folder = drive + "/data_tcspc/20200701"
data_name = ""
musp_12 = np.zeros([len(_nm)])
mua_12 = np.zeros([len(_nm)])
wavelength_id = 0
for wavelength_in_nm in _nm:
    mus, mua = get_mu_from_DTOF(
        wavelength_in_nm, measurement_duration, irf_rho, 
        "", medium_rho, data_name, folder, 
        verbose=False, signal_start=sig_start)
    musp_12[wavelength_id] = (mus)
    mua_12[wavelength_id] = (mua)
    wavelength_id = wavelength_id + 1

## 2nd batch
folder = drive + "/data_tcspc/20200802"
data_name = "_1.5p"
musp_21 = np.zeros([len(_nm)])
mua_21 = np.zeros([len(_nm)])
wavelength_id = 0
for wavelength_in_nm in _nm:
    mus, mua = get_mu_from_DTOF(
        wavelength_in_nm, measurement_duration, irf_rho, 
        "", medium_rho, data_name, folder, 
        verbose=False, signal_start=sig_start)
    musp_21[wavelength_id] = (mus)
    mua_21[wavelength_id] = (mua)
    wavelength_id = wavelength_id + 1
    
folder = drive + "/data_tcspc/20200930"
data_name = "_smofonly"
musp_22 = np.zeros([len(_nm)])
mua_22 = np.zeros([len(_nm)])
wavelength_id = 0
for wavelength_in_nm in _nm:
    mus, mua = get_mu_from_DTOF(
        wavelength_in_nm, measurement_duration, irf_rho, 
        "", medium_rho, data_name, folder, 
        verbose=False, signal_start=sig_start)
    musp_22[wavelength_id] = (mus)
    mua_22[wavelength_id] = (mua)
    wavelength_id = wavelength_id + 1
    
## Divide by zero warnings may occur during fitting.

In [ ]:
matplotlib.rcParams.update({'font.size': 7})
matplotlib.rc('xtick', labelsize=7) 
matplotlib.rc('ytick', labelsize=7)

fig, ax = plt.subplots(1,2, figsize=(6.5,2), dpi=600)

ax[0].plot(_nm_ext, np.asarray(model_musp(_nm_ext, initial_guess_coeffs))/100,
           "--", color="gray",
           label="Tissue model example")

ax[1].plot(water_ref1["(nm)"], water_ref1["(1/cm)"],
           "--", color="gray",
           label="H$_2$O [Segelstein]")
    
fit21 = least_squares(residuals_musp, initial_guess_coeffs, 
                    args=(musp_21, _nm), 
                    bounds=((0, 1e5)), ftol=1e-10)
print("musp500=", fit21.x[0], ", fray=", fit21.x[1], ", bmie=", fit21.x[2])

ax[0].plot(_nm_ext, np.asarray(model_musp(_nm_ext, fit21.x))/100, 
           color="black", linestyle="-", alpha=0.9,
           label="Tissue model fit")

ax[0].plot(_nm, musp_12/100,
           color="tab:blue", marker="+", linestyle="", alpha=0.5,
           label="1st batch")
ax[1].plot(_nm, mua_12/100,
           color="tab:blue", marker="+", linestyle="", alpha=0.5,
           label="1st batch")

ax[0].plot(_nm, musp_21/100,
           color="tab:orange", marker="x", linestyle="", alpha=0.5,
           label="2nd batch, 1st bottle")
ax[1].plot(_nm, mua_21/100,
           color="tab:orange", marker="x", linestyle="", alpha=0.5,
           label="2nd batch, 1st bottle")

ax[0].plot(_nm, musp_22/100,
           color="tab:orange", marker=".", linestyle="", alpha=0.5,
           label="2nd batch, 2st bottle")
ax[1].plot(_nm, mua_22/100,
           color="tab:orange", marker=".", linestyle="", alpha=0.5,
           label="2nd batch, 2st bottle")

ax[0].legend(fontsize=7,frameon=False)
ax[1].legend(fontsize=7,frameon=False)
ax[0].set_xlim(590,980)
ax[1].set_xlim(590,980)
ax[0].set_ylim(10,26)
ax[1].set_ylim(0,6.5e-2)
ax[0].set_xlabel("$\lambda$ [nm]")
ax[0].set_ylabel("$\mu_\mathrm{s}'$ [cm$^{-1}]$")
ax[1].set_xlabel("$\lambda$ [nm]")
ax[1].set_ylabel("$\mu_\mathrm{a}$ [cm$^{-1}]$")
fig.tight_layout()
plt.show()
fig.savefig("smoflipid_spectra.png")
plt.close(fig)